In [161]:
import pandas as pd 
import ast
import re
import numpy as np
# df = pd.read_csv("./data/kickstarter_test.csv")
df = pd.read_csv("/Users/bandy/Downloads/data/kickstarter_train.csv")

df.head(3)
df_copy = df.copy()

In [4]:
df.head()

,id,name,description,description_story,description_risks,rewards,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at
0,3.071031e+08,"""To the things themselves!""","A new dance exploring both my allergy to ""abou...","November 2, 2018 OMG! We're just over halfway ...",We really need your support! Thank you.,"[{""rewards"":""Pledge US$ 25 or more\n\nAbout S$...",dance,9000.0,2018-11-17 04:59:00,US,1,1,https://v2.kickstarter.com/1663805284-vaRunMxL...,2018-10-15 20:59:22,2018-10-09 01:27:56
1,2.143193e+08,Journals 1990-2014,How did we get to the new century? Ride the wa...,Many thanks to my backers for getting us to th...,Here are the steps of my process:\r\n\r\nStep ...,"[{""rewards"":""Pledge US$ 10 or more\n\nAbout US...",publishing,5000.0,2015-04-09 14:39:44,US,1,1,https://v2.kickstarter.com/1664643846-VxuMrMK%...,2015-03-16 14:39:44,2015-02-24 17:29:35
2,7.475064e+07,Bailarina Piano,Hemos conseguido poner en marcha el proyecto B...,-¿Te gustaría disfrutar de la mezcla de la dan...,El principal riesgo es el retrasar el proyecto...,"[{""rewards"":""Pledge €3 or more\n\nAbout S$ 5\n...",music,650.0,2021-05-23 01:36:40,ES,1,0,https://v2.kickstarter.com/1664750650-Kfqx1NI3...,2021-05-03 01:36:40,2021-04-30 22:16:36
3,1.946709e+09,Lily Locksmith full length album,Help Lily Locksmith put out her final recordin...,"Lily Locksmith, with her powerful voice, rich ...",The biggest challange here is the time line. I...,"[{""rewards"":""Pledge SEK 149 or more\n\nAbout S...",music,34000.0,2022-04-14 07:21:20,SE,1,0,https://v2.kickstarter.com/1664734207-Ac%2F11N...,2022-03-20 07:21:20,2022-01-30 12:56:59
4,5.372524e+08,Very Young Composers @ 20!,Help celebrate 20 years of empowering the youn...,"For 20 years now, the Very Young Composers pro...","Live streaming to South Korea, Finland, Venezu...","[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...",music,20000.0,2016-05-06 18:45:32,US,1,1,https://v2.kickstarter.com/1664682609-xfA%2FLS...,2016-04-05 18:45:32,2016-02-24 16:11:49


In [21]:
COUNT = 0

def increment():
    global COUNT
    COUNT = COUNT+1


def clean_rewards(reward_list):
    try:
        tiers = ast.literal_eval(reward_list)
    except SyntaxError as e:
        increment()
        i = len(reward_list) -1
        while reward_list[i] != "{":
            i -= 1
        reward_list = reward_list[:(i-1)]
        reward_list += ']'
    return reward_list

In [22]:
#Clean rewards to remove broken off rewards
df_copy["rewards"] = df_copy["rewards"].apply(lambda x : clean_rewards(x))
df_copy["reward_tiers"] = df_copy["rewards"].apply(lambda x : len(ast.literal_eval(x)))

# Rearange reward_tiers column to beside rewards
cols = df_copy.columns.tolist()
cols = cols[:6] + [cols[-1]] + cols[6:-1]
df_copy = df_copy[cols]
df_copy


Generate all reward amounts for each listing

In [164]:
df_copy["min_reward"] = 0
df_copy["max_reward"] = 0

for i in range(len(df_copy)):
    all_reward_amount = []
    reward = df_copy.iloc[i, 5]
    dict_list = ast.literal_eval(reward)
    # print(dict_list)
    for dictionary in dict_list:
        # print("dictionary item is ", dictionary)

        # if not isinstance(dictionary,  dict):
        #     print("ERROR type is", type(dict))
        values_string = str(dictionary.values())
        # reward_title_USD = re.search(r"(US\$\s)?(\$\s)?\d{1,3}(,\d{1,3})?", values_string) #US$ or $
        reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$
        reward_title_USD = reward_title_SGD
        # reward_title = re.search(r"Pledge US\$ \d{1,3}(,\d{1,3})? or more", values_string)
        if reward_title_SGD or reward_title_USD:
            if reward_title_SGD is not None:
                # print("reward USD title is ", reward_title_USD.group())
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_USD.group())
            else:
                print("ERROR NO SGD")
                # print("reward SGD title is ", reward_title_SGD.group())
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
        else:
            # print("reward title is none")
            pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
            if pledged_without_reward:
                all_reward_amount.append(0)
    # print("ALL REWARD AMOUNT ", all_reward_amount)
    if len(all_reward_amount) > 0:
        df_copy["min_reward"][i] = min(all_reward_amount)
        df_copy["max_reward"][i] = max(all_reward_amount)
    

C:\Users\bandy\AppData\Local\Temp\ipykernel_21828\2523479555.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["min_reward"][i] = min(all_reward_amount)
C:\Users\bandy\AppData\Local\Temp\ipykernel_21828\2523479555.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["max_reward"][i] = max(all_reward_amount)


In [165]:
# Remove rewards that got cut off while scraping and closes the dictionary
def create_rewards_tiers(df):
    # df["rewards"] = df["rewards"].apply(lambda x : clean_rewards(x))
    df["reward_tiers"] = df["rewards"].apply(lambda x : len(ast.literal_eval(x)))
    df = move_reward_tiers(df)
    return df

def create_min_max_reward(df):
    df["min_reward"] = 0
    df["max_reward"] = 0

    for i in range(len(df)):
        all_reward_amount = []
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)

        for dictionary in dict_list:
            values_string = str(dictionary.values())
            reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$
            if reward_title_SGD:
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())
            else:
                 print("Error: No reward in SGD found!")   

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
            else:
                pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
                if pledged_without_reward:
                    all_reward_amount.append(0)
        if len(all_reward_amount) > 0:
            df.loc[i, "min_reward"] = min(all_reward_amount)
            df.loc[i, "max_reward"] = max(all_reward_amount)
            
    print("create min max reward done, now moving columns")
    df = move_min_max_reward(df)
    return df

# Rearange reward_tiers column to the right of rewards
def move_reward_tiers(df):
    cols = df.columns.tolist()
    cols = cols[:6] + [cols[-1]] + cols[6:-1]
    df = df[cols]
    return df

# Rearange min and max reward columns to the right of reward_tiers
def move_min_max_reward(df):
    cols = df.columns.tolist()
    cols = cols[:7] + cols[-2:] + cols[7:-2]
    df = df[cols]
    return df

# Remove backer count in rewards to prevent data leakage
def remove_backers(df):
    new_df = df.copy()
    for i in range(len(df)):
        # print(i)
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        new_list = []

        for dictionary in dict_list:
            values_string = str(dictionary.values())
            match_result = re.search(r"\d+ backer(s)?", values_string) #\nxxx backer
            # print(match_result.group())
            start_index = match_result.span()[0]
            new_reward = values_string[14:start_index] #Remove "dict_values" and backer-count related words
            new_list.append({'rewards': new_reward})

        new_df.iloc[i,5] = str(new_list)
    return new_df


In [166]:
result = create_rewards_tiers(df)
print("Rewards tiers done")


Rewards tiers done


In [167]:
result = create_min_max_reward(result)
print("mins max rewards done")

C:\Users\bandy\AppData\Local\Temp\ipykernel_8472\1181530906.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_reward"] = 0
C:\Users\bandy\AppData\Local\Temp\ipykernel_8472\1181530906.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_reward"] = 0


create min max reward done, now moving columns
mins max rewards done


In [168]:
result.iloc[1,5]

'[{"rewards":"Pledge US$ 10 or more\\n\\nAbout US$ 10\\n\\n\\n\\nEbook of JOURNALS 1990-2014. Heartfelt thanks, and a mention in the acknowledgements section of the JOURNALS.\\n\\n\\nLess\\n\\n\\n\\n\\nEstimated delivery\\nMay 2015\\n\\n\\n\\n\\n\\n\\n39 backers"},{"rewards":"Pledge US$ 25 or more\\n\\nAbout US$ 25\\n\\n\\n\\nSame as the $10 level, plus free ebooks of nine other Rucker Transreal Books titles: TRANSREAL TRILOGY, ALL THE VISIONS, MATHEMATICIANS IN LOVE, JIM AND THE FLIMS, THE BIG AHA, TURING & BURROUGHS, COLLECTED ESSAYS, COMPLETE STORIES, and THE HOLLOW EARTH.\\n\\n\\nLess\\n\\n\\n\\n\\nEstimated delivery\\nMay 2015\\n\\n\\n\\n\\n\\n\\n17 backers"},{"rewards":"Pledge US$ 50 or more\\n\\nAbout US$ 50\\n\\n\\n\\nAll of the above, plus a paperback collector\'s edition of JOURNALS 1990-2014. Book will include a signed and numbered bookplate designed by me. Book size 6.7\\" x 9.6\\" and 820 pages.\\n\\n\\nLess\\n\\n\\n\\n\\nEstimated delivery\\nJun 2015\\n\\n\\nShips to\\nAn

In [169]:
result = remove_backers(result)
print("remove backers done")

remove backers done


In [158]:
result.iloc[0,5]

'[{\'rewards\': \'Pledge US$ 25 or more\\\\n\\\\nAbout S$ 36\\\\n\\\\n\\\\n\\\\nFRIEND\\\\n\\\\n\\\\nA personal shout-out on Facebook and a listing in the program as a supporter of the project.\\\\n\\\\n\\\\nLess\\\\n\\\\n\\\\n\\\\n\\\\nEstimated delivery\\\\nDec 2018\\\\n\\\\n\\\\n\\\\n\\\\n\\\\n\\\\n\'}, {\'rewards\': \'Pledge US$ 50 or more\\\\n\\\\nAbout S$ 72\\\\n\\\\n\\\\n\\\\nDONOR\\\\n\\\\n\\\\nTwo video artifacts from the rehearsal process for “To The Things Themselves!” – a clip of one of the source movement improvisations, and a clip of a construction made using that source video – selected especially for you and sent via email.\\\\n\\\\n\\\\nLess\\\\n\\\\n\\\\n\\\\n\\\\nEstimated delivery\\\\nDec 2018\\\\n\\\\n\\\\n\\\\n\\\\n\\\\n\\\\n\'}, {\'rewards\': \'Pledge US$ 100 or more\\\\n\\\\nAbout S$ 143\\\\n\\\\n\\\\n\\\\nCONTRIBUTOR\\\\n\\\\n\\\\nAll of the above, plus: An archival clip of one of Neil’s 1994 source movement improvisations for “Not-About-AIDS-Dance” (my he was 

In [137]:
def remove_backers(df):
    new_df = df.copy()
    for i in range(len(df)):
        # print(i)
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        new_list = []
        for dictionary in dict_list:
            values_string = str(dictionary.values())
            match_result = re.search(r"\d+ backer(s)?", values_string) #\nxxx backer
            # print(match_result.group())
            start_index = match_result.span()[0]
            new_reward = values_string[14:start_index] #Remove backer-count related words to prevent data leakage
            new_list.append({"rewards": new_reward})

        new_df.iloc[i,5] = str(new_list)
    return new_df